In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

df = pd.read_excel('/home/ameer/Documents/pre-post-sale/basetags_annotated.ods')
df.head()

2023-03-02 10:07:55.571586: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-02 10:07:55.609289: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-02 10:07:55.609310: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


,Unnamed: 0,conversations_content,base_tags
0,NaN,[‘ Sharanya Munsi sharanyamunsi@gmail.com 891...,pre_sale
1,NaN,[‘I need to make bangle of my own design Conve...,pre_sale
2,NaN,[‘Conversation was to Saba Shaikh by Intelli...,pre_sale
3,NaN,[‘How to check EMI option? Deepa,pre_sale
4,NaN,[‘please delete my account’],oth


In [2]:
pd.set_option('display.width', 10)
pd.set_option("max_colwidth", 1000)

In [4]:
def preprocess(df):
    df['conversations_content'] = df['conversations_content'].astype(str).str.replace('[','',regex = True)
    df['conversations_content'] = df['conversations_content'].str.replace(']','',regex = True)
    df['conversations_content'] = df['conversations_content'].str.replace('/','',regex = True)
    df['conversations_content'] = df['conversations_content'].str.replace('`','')
    df['conversations_content'] = df['conversations_content'].str.replace("',",'')
    df['conversations_content'] = df['conversations_content'].str.replace('",','')
    df['conversations_content'] = df['conversations_content'].str.replace("'",'')
    df['conversations_content'] = df['conversations_content'].str.replace('"','')
    df['conversations_content'] = df['conversations_content'].str.replace(',','.')
    #df1['conversations_content'] = df1['conversations_content'].str.replace(r'\..','.')
    #df1['conversations_content'] = df1['conversations_content'].str.replace('ill','i will')
    df['conversations_content'] = df['conversations_content'].str.replace(r'-','.')
    df['conversations_content'] = df['conversations_content'].str.replace(r'\\n','',regex = True)
    df['conversations_content'] = df['conversations_content'].str.replace(r'\\t','',regex = True)
    #df1['conversations_content'] = df1['conversations_content'].str.replace('  ',' ')
    
    return df

In [4]:
df =  preprocess(df)

In [3]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [6]:
df.shape

(1953, 2)

In [7]:
df.sample(5)

,conversations_content,base_tags
52,‘Regarding GWD’,oth
668,Welcome to CaratLane . A Tanishq Partnership. We are Indias largest and most trusted online jeweller. Thank you for contacting CaratLane! Hope you are safe and doing good. This is Gourav here. How can I help you? Hi Gourav I want to avail the Kisht option for product in my cart Can you let me know how to avail the same? ? Hi Sure! We do have an Option for EMI! It will be via Liquiloans and it is a cibil score based EMI! Post a credit score check. the EMI will be provided. It is also a downpayment based EMI! Ok We have interest free EMI via LIQUILOANS on purchase of on or above Rs.15000 and the minimum salary in hand should be Rs.20.000The tenures are as follows: 8 months. (6 EMI months + 2 months down payment) & processing fee of 4%+gst on processing fee12 months. (8 EMI months + 4 months down payment) & processing fee of 4%+gst on processing fee10 months. (8 EMI months + 2 months down payment) & processing fee of 5%+gst on processing fee. The details required are:. 2...,pre_sale
28,‘I want to redeem my points’,pre_sale
1840,Hi Hadeel mazenmahmood31@gmail.com May i ask you is there return option available? Conversation was assigned to Inamul H by IntelliAssign If i purchase it online? Thank you for contacting CaratLane. A Tanishq Partnership. Hope your doing good ? This is Inamul&nbsp; How can i assist you today..? yeah and hope you too We have return options available for both gold and solitaire jewelery is there return option available? If i purchase it through online? Are you looking for a purchase at the moment right now? No. but i will br ordering it soon so wanted to confirm first that about return because i couldnt read it anywhere That is really great We have a 15 day exchange and a 15 day money back policies were we will be giving a full invoice value exchange and return for orders worth 1.00.000rs in gold and in case of solitaire it is upto 5.00.000rs Yeah thats what I was asking but I will not be purchasing upto that value I am sorry if my earlier statement is not clear enough. if you ord...,post_sale
735,‘I want to cancel my order’,post_sale


In [8]:
df.dropna(inplace=True)

In [9]:
possible_labels = df.base_tags.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'pre_sale': 0, 'oth': 1, 'post_sale': 2}

In [10]:
df['label'] = df.base_tags.replace(label_dict)

In [11]:
df.head()

,conversations_content,base_tags,label
0,‘ Sharanya Munsi sharanyamunsi@gmail.com 8910008094 Conversation was to Abdul Salam Khan by IntelliAssign This is Abdul here! glad to assist you&nbsp; I need with updating my birthday date It is 7th vember I plan on making a purchase and would like to receive the additional birthday discount let me know if that is possible because I just got a call saying I cant as my coupon has expired For that you need to mail us on Contactus@caratlane.com Is there anything else that I may you with? If i update my date will the discount be made available this month? t possible now \ryou should have informed us one month back Is there anything else that I may you with? you for your time! It was my pleasure to assist you. I hope I answered all your queries to your satisfaction. Kindly rate the after you close the .Have a good day :)Looking forward to serving you again! :) Conversation was by Abdul Salam Khan’,pre_sale,0
1,‘I need to make bangle of my own design Conversation reopened due to new incoming message May I know the process Assigned to group Presale . Chat by Kinjal Adodra Conversation was assigned to Sudhanshu Panda by IntelliAssign We appreciate you contacting CaratLane.Hope you are doing well!This is Sudhanshu&nbsp;More than happy to help you :) nan. May I know how much it will cost If I have this one to make Kindly allow me few minutes for the same please . Kindly let me know the bangle size along with gold type . and which quality diamonds you want . Tell me . what the quality of diamonds means? 18K gold 2.3 size And screw is also needed Hi Kindly allow me some time to ask with our customization team for the same . Before that shall I send you a similar design of the same which is available at our Caratlane . Do send the design Assisted .&gt;connected on WhatsApp.&gt;will follow up. As we are connected on WhatsApp . I am closing this chat from here . For any further queries . you ...,pre_sale,0
2,‘Conversation was to Saba Shaikh by IntelliAssign. Do we have a option to buy a voucher ? Welcome to Caratlane A Tanishq Partnership&nbsp; My name is Saba \r I would like to inform you that as of now we stop selling voucher I was willing to gift a voucher sort of thing to someone I do understand but i am so sorry Conversation was by Saba Shaikh’,pre_sale,0
3,‘How to check EMI option? Deepa deepabhat613@gmail.com 8792135431 Conversation was assigned to HItesh Makwana by IntelliAssign How to check EMI option Hi ! This is Hitesh.\rThank you for contacting CaratLane. A Tanishq Partnership.\rHope you and your family are safe Please confirm do you wish to purchase new product call the customer { cx wants to purchase some new products she requested to call her back on weekends } Conversation was marked resolved by HItesh Makwana’,pre_sale,0
4,‘please delete my account’,oth,1


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1951 entries, 0 to 1952
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   conversations_content  1951 non-null   object
 1   base_tags              1951 non-null   object
 2   label                  1951 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 61.0+ KB


In [13]:
df['label'] = df['label'].astype(int)

In [14]:
df.head()

,conversations_content,base_tags,label
0,‘ Sharanya Munsi sharanyamunsi@gmail.com 8910008094 Conversation was to Abdul Salam Khan by IntelliAssign This is Abdul here! glad to assist you&nbsp; I need with updating my birthday date It is 7th vember I plan on making a purchase and would like to receive the additional birthday discount let me know if that is possible because I just got a call saying I cant as my coupon has expired For that you need to mail us on Contactus@caratlane.com Is there anything else that I may you with? If i update my date will the discount be made available this month? t possible now \ryou should have informed us one month back Is there anything else that I may you with? you for your time! It was my pleasure to assist you. I hope I answered all your queries to your satisfaction. Kindly rate the after you close the .Have a good day :)Looking forward to serving you again! :) Conversation was by Abdul Salam Khan’,pre_sale,0
1,‘I need to make bangle of my own design Conversation reopened due to new incoming message May I know the process Assigned to group Presale . Chat by Kinjal Adodra Conversation was assigned to Sudhanshu Panda by IntelliAssign We appreciate you contacting CaratLane.Hope you are doing well!This is Sudhanshu&nbsp;More than happy to help you :) nan. May I know how much it will cost If I have this one to make Kindly allow me few minutes for the same please . Kindly let me know the bangle size along with gold type . and which quality diamonds you want . Tell me . what the quality of diamonds means? 18K gold 2.3 size And screw is also needed Hi Kindly allow me some time to ask with our customization team for the same . Before that shall I send you a similar design of the same which is available at our Caratlane . Do send the design Assisted .&gt;connected on WhatsApp.&gt;will follow up. As we are connected on WhatsApp . I am closing this chat from here . For any further queries . you ...,pre_sale,0
2,‘Conversation was to Saba Shaikh by IntelliAssign. Do we have a option to buy a voucher ? Welcome to Caratlane A Tanishq Partnership&nbsp; My name is Saba \r I would like to inform you that as of now we stop selling voucher I was willing to gift a voucher sort of thing to someone I do understand but i am so sorry Conversation was by Saba Shaikh’,pre_sale,0
3,‘How to check EMI option? Deepa deepabhat613@gmail.com 8792135431 Conversation was assigned to HItesh Makwana by IntelliAssign How to check EMI option Hi ! This is Hitesh.\rThank you for contacting CaratLane. A Tanishq Partnership.\rHope you and your family are safe Please confirm do you wish to purchase new product call the customer { cx wants to purchase some new products she requested to call her back on weekends } Conversation was marked resolved by HItesh Makwana’,pre_sale,0
4,‘please delete my account’,oth,1


In [15]:
df['label'].value_counts()

2    838
0    660
1    453
Name: label, dtype: int64

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', 
                                          do_lower_case=True)

In [17]:
token = tokenizer.encode_plus(
    df['conversations_content'].iloc[0], 
    max_length=512, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)

2022-11-28 11:17:41.378082: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-28 11:17:41.378111: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-28 11:17:41.378129: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (CLLAP-CHN-1343): /proc/driver/nvidia/version does not exist
2022-11-28 11:17:41.379222: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
X_input_ids = np.zeros((len(df), 512))
X_attn_masks = np.zeros((len(df), 512))

In [19]:
X_input_ids.shape

(1951, 512)

In [20]:
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['conversations_content'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=512, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [21]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [22]:
X_input_ids

array([[ 101.,  786.,  188., ...,    0.,    0.,    0.],
       [ 101.,  786.,  178., ...,    0.,    0.,    0.],
       [ 101.,  786., 3771., ...,    0.,    0.,    0.],
       ...,
       [ 101., 3771., 1108., ...,    0.,    0.,    0.],
       [ 101., 3771., 1108., ...,    0.,    0.,    0.],
       [ 101., 3771., 1108., ...,    0.,    0.,    0.]])

In [23]:
labels = np.zeros((len(df), 3))
labels.shape

(1951, 3)

In [24]:
labels[np.arange(len(df)), df['label'].values] = 1

In [25]:
labels

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [26]:
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))

In [27]:
dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(512,), dtype=tf.float64, name=None), TensorSpec(shape=(512,), dtype=tf.float64, name=None), TensorSpec(shape=(3,), dtype=tf.float64, name=None))>

In [28]:
def ClassMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    },labels

In [29]:
dataset = dataset.map(ClassMapFunction) # converting to required format for tensorflow dataset

In [30]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.float64, name=None)}, TensorSpec(shape=(3,), dtype=tf.float64, name=None))>

In [31]:
dataset = dataset.shuffle(1538).batch(2, drop_remainder=True) # batch size, drop any left out tensor

In [32]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(2, 512), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(2, 512), dtype=tf.float64, name=None)}, TensorSpec(shape=(2, 3), dtype=tf.float64, name=None))>

In [33]:
p = 0.95
train_size = int((len(df)//1.2)*p) # for each 2 batch of data we will have len(df)//2 samples, take 95% of that for train.


In [34]:
train_size

1543

In [35]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [36]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(2, 512), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(2, 512), dtype=tf.float64, name=None)}, TensorSpec(shape=(2, 3), dtype=tf.float64, name=None))>

In [37]:
from transformers import TFBertModel

In [38]:
model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [39]:
# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(512,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(512,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(3, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

class_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
class_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [40]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [41]:
class_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [42]:
hist = class_model.fit(train_dataset,validation_data=val_dataset,epochs=5)

Epoch 1/5


2022-11-28 11:18:06.634424: E tensorflow/core/framework/node_def_util.cc:630] NodeDef mentions attribute epsilon which is not in the op definition: Op<name=_MklFusedBatchMatMulV2; signature=x:T, y:T, args:num_args*T -> output:T; attr=T:type,allowed=[DT_BFLOAT16, DT_FLOAT]; attr=adj_x:bool,default=false; attr=adj_y:bool,default=false; attr=num_args:int,min=0; attr=fused_ops:list(string),default=[]> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node model/bert/encoder/layer_._0/attention/self/ArithmeticOptimizer/AddOpsRewrite_add_1}}


975/975 [==============================] - 6663s 7s/step - loss: 0.5046 - accuracy: 0.7990
Epoch 2/5
975/975 [==============================] - 6704s 7s/step - loss: 0.1878 - accuracy: 0.9431
Epoch 3/5
975/975 [==============================] - 7495s 8s/step - loss: 0.1095 - accuracy: 0.9744
Epoch 4/5
975/975 [==============================] - 9535s 10s/step - loss: 0.0955 - accuracy: 0.9728
Epoch 5/5
975/975 [==============================] - 63111s 65s/step - loss: 0.0842 - accuracy: 0.9764


In [43]:
class_model.save('class_model_5')

INFO:tensorflow:Assets written to: class_model_5/assets


INFO:tensorflow:Assets written to: class_model_5/assets


In [1]:
import pickle

In [ ]:
#save the model to disk
filename = 'class_model_5.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

In [5]:
class_model = pickle.load(open('class_model_5.sav', 'rb'))   ##tf.keras.models.load_model('class_model_5')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=512, 
        truncation=True, 
        padding='max_length',  
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['pre_sale', 'oth', 'post_sale']):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]


2023-03-02 10:08:43.386433: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-02 10:08:43.386774: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-02 10:08:43.386960: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (CLLAP-CHN-1343): /proc/driver/nvidia/version does not exist
2023-03-02 10:08:43.388080: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [45]:
input_text = 'Conversation was marked resolved by Sudhanshu  Panda'

In [46]:
#input_text = input('Enter text here: ')
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(class_model, processed_data=processed_data)
print(f"Predicted Class: {result}")

1/1 [==============================] - 2s 2s/step
Predicted Class: oth


In [6]:
def createdataframe(filepath):
    df = pd.read_excel(filepath)
    df = preprocess(df)
    df['tag']=''
    return df
def predict(input_text):
    processed_data = prepare_data(input_text, tokenizer)
    result = make_prediction(class_model, processed_data=processed_data)
    return result

In [30]:
df = createdataframe('/home/ameer/Documents/pre-post-sale/subtags_output6.xlsx')

In [31]:
import swifter

In [32]:
df['tag'] = df['conversations_content'].swifter.apply(lambda x: predict(x))

Pandas Apply:   0%|          | 0/400 [00:00<?, ?it/s]

1/1 [==============================] - 1s 909ms/step


1/1 [==============================] - 1s 906ms/step


In [16]:
df

,Unnamed: 0.1,Unnamed: 0,customer_id,conversation_id,interaction_id,chat_id,actor_type,created_time,resolved,channel_id,message_parts,conversations_content,content_array,subtag,tag
0,0,111675,B471593D-FD91-45F9-A113-3A4943C9C423,8cbffe26-78d6-4587-8c01-8fe533c05d9b,8cbffe26-78d6-4587-8c01-8fe533c05d9b-167637584...,e94b0806-b9c6-48f4-b5fc-c973cedb0221,agent,2023-02-14 14:49:13.394,1,d86f499f-092b-4531-8803-608686e36869,Conversation was marked resolved by Neel B,Hi Conversation was assigned to Harsh Mehta by...,"[{'visitor:Hi'}, {'agent:Conversation was assi...",price break up,pre_sale
1,1,107268,39ba1416-be75-4b43-8e66-ea2efd9e587f,a05d1a64-7b5e-436c-b960-c6c6f0cb62b3,a05d1a64-7b5e-436c-b960-c6c6f0cb62b3-167636493...,9a16286c-b560-42ea-b359-c1219105a416,agent,2023-02-14 14:47:29.256,1,d86f499f-092b-4531-8803-608686e36869,Conversation was marked resolved by Priyanka B...,Conversation was reopened by Mainak Mallick hi...,[{'visitor:Conversation was reopened by Mainak...,none,oth
2,2,70682,20908797-e4f2-4482-803f-9bd8cd073c75,f6df81b5-580d-4c52-bb64-f4694b3542dc,f6df81b5-580d-4c52-bb64-f4694b3542dc-167638488...,31988e72-62f4-46e0-aacd-ec84e8a3cce5,agent,2023-02-14 14:45:41.184,1,1925a070-b98f-4ab3-9812-50fe474f0b48,Conversation was marked resolved by Saba Shaikh,Hi Pallavi pallavi.pnd@gmail.com +918120386208...,"[{'visitor:Hi'}, {'visitor:Pallavi'}, {'visito...",jewellery selection,pre_sale
3,3,26678,DE5BC928-21B2-4FE8-89EC-16CE41C1558B,cc7aa350-405d-4daa-ae87-958c3886587b,cc7aa350-405d-4daa-ae87-958c3886587b-167638293...,6b846b8a-087b-4e86-832e-8d0b581580b2,agent,2023-02-14 14:45:36.089,1,1925a070-b98f-4ab3-9812-50fe474f0b48,Conversation was marked resolved by Priyanka B...,Hi Conversation was assigned to Priyanka Biswa...,"[{'visitor:Hi'}, {'agent:Conversation was assi...",none,oth
4,4,74837,d677bd69-94f0-44a7-9cd5-888f5799cad1,f2b28644-c8aa-406f-b130-d20562c6691c,f2b28644-c8aa-406f-b130-d20562c6691c-167637877...,7f1d6cb4-ff69-4d9c-928a-3dc26e8432ad,agent,2023-02-14 14:27:06.317,1,1925a070-b98f-4ab3-9812-50fe474f0b48,Conversation was marked resolved by Mohammed A,Hii Rejeena rejeenajoy99@gmail.com 8893620820 ...,"[{'visitor:Hii'}, {'visitor:Rejeena'}, {'visit...",Lifetime exchange,pre_sale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,83512,54317279-F35E-4D72-BD1C-CEABB4D3EED1,109e5ff6-4db1-44a8-b39f-b501b7ffaf0b,109e5ff6-4db1-44a8-b39f-b501b7ffaf0b-167628832...,56b67fc5-0a97-4b01-98d5-ad0d365e9360,agent,2023-02-13 12:43:11.000,1,d86f499f-092b-4531-8803-608686e36869,Conversation was marked resolved by Salma S,Conversation was reopened by Deeba Farah When ...,[{'visitor:Conversation was reopened by Deeba ...,Lifetime exchange,post_sale
196,196,41321,b0239c90-d470-431e-96de-6d620f6a9df5,8da5e35c-114e-4c3e-bfb3-4f5f82e4aede,8da5e35c-114e-4c3e-bfb3-4f5f82e4aede-167622149...,37d8cb06-6ead-46da-ae5b-c81cf69fb8e6,agent,2023-02-13 12:42:46.442,1,d86f499f-092b-4531-8803-608686e36869,Conversation was marked resolved by Apurva Shetty,i order another ring in exchange of this broke...,[{'visitor:i order another ring in exchange of...,Lifetime exchange,pre_sale
197,197,4430,38555542-c0be-4b45-b304-d7d8cb218d8f,a567c2d1-7894-4df2-84a7-bf9befba37c1,a567c2d1-7894-4df2-84a7-bf9befba37c1-167628561...,5106777f-849b-41f6-bf04-47eaa0af937f,agent,2023-02-13 12:41:51.667,1,d86f499f-092b-4531-8803-608686e36869,Conversation was marked resolved by Siddhi d,I missplace 1 earring of this order no . Is it...,[{'visitor:I missplace 1 earring of this order...,Old gold exchange,post_sale
198,198,118409,b25ead53-270a-4de5-a46d-4a2b57fe3297,5c50a28a-0f26-42e8-82c7-6fd42076a5a3,5c50a28a-0f26-42e8-82c7-6fd42076a5a3-167628909...,871666ba-153c-46d7-a9f9-51124a59ea59,agent,2023-02-13 12:36:41.364,1,d86f499f-092b-4531-8803-608686e36869,Conversation was marked resolved by Salma S,hi Akash chatterjee chatterjrr.akash1522@gmai...,"[{'visitor:hi'}, {'visitor:Akash chatterjee '}...",rush delivery,post_sale


In [33]:
df.to_excel('tagssubtags_output6.xlsx')

In [ ]:
from keybert import KeyBERT

In [ ]:
kw_model = KeyBERT()

In [ ]:
def extract_keys(text):
    keywords = kw_model.extract_keywords(text)
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 2), stop_words='english', top_n = 6, use_maxsum=False, diversity=0.4, nr_candidates=20)
    return keywords

In [ ]:
def keyonly(lst):
    k = []
    for i in range(0, len(lst)):
        temp = (lst[i][0])
        k.append(temp)
    return k 

In [ ]:
df1['keywords'] = df1['conversations_content'].apply(lambda x: extract_keys(x))

In [ ]:
df1['keywords'] = df1['keywords'].apply(lambda x: keyonly(x))

In [ ]:
df1

In [ ]:
#file_name = 'pre-post.xlsx'
#df.to_excel(file_name)
#print('DataFrame is written to Excel File successfully.')